## Deliverable 2. Create a Customer Travel Destinations Map.

In [16]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [17]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mfou,CM,3.7159,11.6427,70.02,97,100,3.87,light rain
1,1,Miguel Auza,MX,24.2933,-103.4503,76.80,32,93,12.50,overcast clouds
2,2,Tuktoyaktuk,CA,69.4541,-133.0374,14.00,85,100,8.05,fog
3,3,Albany,US,42.6001,-73.9662,54.25,85,100,1.86,overcast clouds
4,4,Salalah,OM,17.0151,54.0924,84.29,70,20,4.61,few clouds


In [18]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [19]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mfou,CM,3.7159,11.6427,70.02,97,100,3.87,light rain
1,1,Miguel Auza,MX,24.2933,-103.4503,76.80,32,93,12.50,overcast clouds
4,4,Salalah,OM,17.0151,54.0924,84.29,70,20,4.61,few clouds
6,6,Angoche,MZ,-16.2325,39.9086,76.91,87,100,7.67,overcast clouds
7,7,Saint-Philippe,RE,-21.3585,55.7679,73.22,82,37,11.36,light rain
9,9,Sao Filipe,CV,14.8961,-24.4956,75.81,69,20,14.14,few clouds
11,11,Jalu,LY,29.0331,21.5482,84.31,13,86,14.45,overcast clouds
12,12,Barpeta,IN,26.3167,91.0000,77.47,73,0,5.66,light rain
13,13,Saint-Pierre,RE,-21.3393,55.4781,73.29,73,0,4.61,clear sky
15,15,Kapaa,US,22.0752,-159.3190,76.98,91,100,18.41,moderate rain


In [20]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [21]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mfou,CM,3.7159,11.6427,70.02,97,100,3.87,light rain
1,1,Miguel Auza,MX,24.2933,-103.4503,76.80,32,93,12.50,overcast clouds
4,4,Salalah,OM,17.0151,54.0924,84.29,70,20,4.61,few clouds
6,6,Angoche,MZ,-16.2325,39.9086,76.91,87,100,7.67,overcast clouds
7,7,Saint-Philippe,RE,-21.3585,55.7679,73.22,82,37,11.36,light rain
9,9,Sao Filipe,CV,14.8961,-24.4956,75.81,69,20,14.14,few clouds
11,11,Jalu,LY,29.0331,21.5482,84.31,13,86,14.45,overcast clouds
12,12,Barpeta,IN,26.3167,91.0000,77.47,73,0,5.66,light rain
13,13,Saint-Pierre,RE,-21.3393,55.4781,73.29,73,0,4.61,clear sky
15,15,Kapaa,US,22.0752,-159.3190,76.98,91,100,18.41,moderate rain


In [22]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mfou,CM,70.02,light rain,3.7159,11.6427,
1,Miguel Auza,MX,76.80,overcast clouds,24.2933,-103.4503,
4,Salalah,OM,84.29,few clouds,17.0151,54.0924,
6,Angoche,MZ,76.91,overcast clouds,-16.2325,39.9086,
7,Saint-Philippe,RE,73.22,light rain,-21.3585,55.7679,
9,Sao Filipe,CV,75.81,few clouds,14.8961,-24.4956,
11,Jalu,LY,84.31,overcast clouds,29.0331,21.5482,
12,Barpeta,IN,77.47,light rain,26.3167,91.0000,
13,Saint-Pierre,RE,73.29,clear sky,-21.3393,55.4781,
15,Kapaa,US,76.98,moderate rain,22.0752,-159.3190,


In [23]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
   
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [24]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mfou,CM,70.02,light rain,3.7159,11.6427,Maison Du Juge Noah
1,Miguel Auza,MX,76.80,overcast clouds,24.2933,-103.4503,Hotel Del Centro
4,Salalah,OM,84.29,few clouds,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
6,Angoche,MZ,76.91,overcast clouds,-16.2325,39.9086,Guest House HESADA
7,Saint-Philippe,RE,73.22,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
9,Sao Filipe,CV,75.81,few clouds,14.8961,-24.4956,Hotel Xaguate
11,Jalu,LY,84.31,overcast clouds,29.0331,21.5482,صاحبة الفخامة
12,Barpeta,IN,77.47,light rain,26.3167,91.0000,Baba Ka Dhaba Hotel
13,Saint-Pierre,RE,73.29,clear sky,-21.3393,55.4781,Lindsey Hôtel
15,Kapaa,US,76.98,moderate rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort


In [25]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [33]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [34]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))